# Qwen2-7B-Instruct-LoRA技术微调

## 步骤1 导入相关包

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

#模型下载
from modelscope import snapshot_download
model_dir = snapshot_download('qwen/Qwen2-7B-Instruct')

Downloading: 100%|██████████| 663/663 [00:00<00:00, 1.77kB/s]
Downloading: 100%|██████████| 48.0/48.0 [00:00<00:00, 115B/s]
Downloading: 100%|██████████| 243/243 [00:00<00:00, 593B/s]
Downloading: 100%|██████████| 11.1k/11.1k [00:00<00:00, 28.5kB/s]
Downloading: 100%|██████████| 1.59M/1.59M [00:00<00:00, 2.77MB/s]
Downloading: 100%|██████████| 3.67G/3.67G [04:25<00:00, 14.9MB/s]
Downloading:  64%|██████▍   | 2.30G/3.60G [02:33<01:08, 20.5MB/s]2024-09-04 20:47:37,938 - modelscope - WARNING - Downloading: /root/autodl-tmp/cache/modelscope/hub/._____temp/qwen/Qwen2-7B-Instruct/model-00002-of-00004.safetensors failed, reason: ('Connection broken: IncompleteRead(6107354 bytes read, 161664806 more expected)', IncompleteRead(6107354 bytes read, 161664806 more expected)) will retry
Downloading: 3.60GB [03:55, 16.4MB/s]                            
Downloading: 100%|██████████| 3.60G/3.60G [04:14<00:00, 15.2MB/s]
Downloading: 100%|██████████| 3.31G/3.31G [03:43<00:00, 15.9MB/s]
Downloading: 100%

## 步骤2 加载数据集

In [2]:
ds = load_dataset("uITimeCia/Zhenhuan",split='train[:1000]')
ds

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 1000
})

In [3]:
ds[:1]

{'instruction': ['小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——'],
 'input': [''],
 'output': ['嘘——都说许愿说破是不灵的。']}

## 步骤3 数据集预处理

In [4]:
#加载本地模型，提前下载到本地
tokenizer = AutoTokenizer.from_pretrained("/root/autodl-tmp/cache/modelscope/hub/qwen/Qwen2-7B-Instruct")
tokenizer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Qwen2TokenizerFast(name_or_path='/root/autodl-tmp/cache/modelscope/hub/qwen/Qwen2-7B-Instruct', vocab_size=151643, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

补充说明：
- Qwen2 采用的Prompt Template格式如下：
```text
<|im_start|>system\nSYSTEM_PROMPT<|im_end|>\n<|im_start|>user\nHUMAN_PROMPT<|im_end|>\n<|im_start|>assistant\nAI_OUT<|im_end|>
```
- 例如：
```text
<|im_start|>system
现在你要扮演皇帝身边的女人--甄嬛<|im_end|>
<|im_start|>user
小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——<|im_end|>
<|im_start|>assistant
嘘——都说许愿说破是不灵的。<|im_end|>
```


In [5]:
def process_func(example):
    MAX_LENGTH = 384    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f"<|im_start|>system\n现在你要扮演皇帝身边的女人--甄嬛<|im_end|>\n<|im_start|>user\n{example['instruction'] + example['input']}<|im_end|>\n<|im_start|>assistant\n", add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [6]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1000
})

In [7]:
tokenizer.decode(tokenized_id[0]['input_ids'])

'<|im_start|>system\n现在你要扮演皇帝身边的女人--甄嬛<|im_end|>\n<|im_start|>user\n小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——<|im_end|>\n<|im_start|>assistant\n嘘——都说许愿说破是不灵的。<|endoftext|>'

In [8]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[1]["labels"])))

'你们俩话太多了，我该和温太医要一剂药，好好治治你们。<|endoftext|>'

## 步骤4 创建模型

In [9]:
import torch
model = AutoModelForCausalLM.from_pretrained("/root/autodl-tmp/cache/modelscope/hub/qwen/Qwen2-7B-Instruct", 
                    torch_dtype=torch.bfloat16,device_map="auto")
model

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Lin

In [10]:
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法
model.dtype # 查看精度

torch.bfloat16

### 1、PEFT 步骤1 配置文件

In [11]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'down_proj', 'up_proj', 'o_proj', 'gate_proj', 'v_proj', 'q_proj', 'k_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

### 2、PEFT 步骤2 创建模型

In [12]:
model = get_peft_model(model, config)
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 3584)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2SdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=3584, out_features=3584, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3584, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [13]:
# 打印出模型中可训练参数的数量
model.print_trainable_parameters()

trainable params: 20,185,088 || all params: 7,635,801,600 || trainable%: 0.2643


## 步骤5 配置训练参数

In [14]:
args = TrainingArguments(
    output_dir="/root/autodl-tmp/cache/finetuning/Qwen2_7B_instruct_lora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=3,
    save_steps=100, # 为了快速演示，这里设置10，建议你设置成100
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True
)


## 步骤6 创建训练器

In [15]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

## 步骤7 模型训练

In [16]:
trainer.train()

[2024-09-04 21:05:13,735] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/root/autodl-tmp/conda/envs/transformers/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,3.583000
20,3.089600
30,3.054100
40,2.941000
50,2.915600
60,2.867400
70,2.820600
80,2.608000
90,2.643100
100,2.552300


/root/autodl-tmp/conda/envs/transformers/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=186, training_loss=2.5958716330989713, metrics={'train_runtime': 313.3396, 'train_samples_per_second': 9.574, 'train_steps_per_second': 0.594, 'total_flos': 1.1635669372981248e+16, 'train_loss': 2.5958716330989713, 'epoch': 2.976})

## 步骤8 模型推理

In [18]:
# 合并模型
merged_model = model.merge_and_unload()
# 保存模型
merge_model_path = "/root/autodl-tmp/cache/finetuning/Qwen2_7B_instruct_lora_merged"
merged_model.save_pretrained(merge_model_path, safe_serialization=True)
tokenizer.save_pretrained(merge_model_path)

('/root/autodl-tmp/cache/finetuning/Qwen2_7B_instruct_lora_merged/tokenizer_config.json',
 '/root/autodl-tmp/cache/finetuning/Qwen2_7B_instruct_lora_merged/special_tokens_map.json',
 '/root/autodl-tmp/cache/finetuning/Qwen2_7B_instruct_lora_merged/vocab.json',
 '/root/autodl-tmp/cache/finetuning/Qwen2_7B_instruct_lora_merged/merges.txt',
 '/root/autodl-tmp/cache/finetuning/Qwen2_7B_instruct_lora_merged/added_tokens.json',
 '/root/autodl-tmp/cache/finetuning/Qwen2_7B_instruct_lora_merged/tokenizer.json')

In [17]:
# 模型推理
prompt = "你是谁？"
messages = [
    #{"role": "system", "content": "现在你要扮演皇帝身边的女人--甄嬛"},
    {"role": "user", "content": "假设你是皇帝身边的女人--甄嬛。"},
    {"role": "user", "content": prompt}
]
inputs = tokenizer.apply_chat_template(messages,add_generation_prompt=True,tokenize=True,return_tensors="pt",return_dict=True).to('cuda')

gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))


/root/autodl-tmp/conda/envs/transformers/lib/python3.11/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


奴婢是新入宫的嫔妃，见过王爷。
